In [3]:
# Connect to google drive
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [4]:
# Cloud authentication
from google.colab import auth
auth.authenticate_user()

ModuleNotFoundError: No module named 'google.colab'

In [5]:
# Import, authenticate and initialize the Earth Engine library
import ee
ee.Authenticate()
ee.Initialize()

ModuleNotFoundError: No module named 'StringIO'

In [6]:
# Import modules
import pandas as pd

In [ ]:
csv_path = '/content/drive/My Drive/SOC_LDSF.csv'
df = pd.read_csv(csv_path)
df.vegstructure = pd.Categorical(df.vegstructure)
df.vegstructure = df.vegstructure.astype('category').cat.codes + 1
df

In [ ]:
def cloudMask(image):
  
  qa = image.select('QA_PIXEL')
  dilated = 1 << 1
  cirrus = 1 << 2
  cloud = 1 << 3
  shadow = 1 << 4

  mask = ((qa.bitwise_and(dilated) == 0) & (qa.bitwise_and(cirrus) == 0) & (qa.bitwise_and(cloud) == 0) & (qa.bitwise_and(shadow) == 0))

  SR = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  image = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  image = image.addBands(SR, None, True)
    
  return image.updateMask(mask)

def addEVI(image):
  evi = image.expression( '2.5 * ( (NIR-RED) / (NIR + 6*RED - 7.5*BLUE + 1))', {
      'NIR': image.select('SR_B5'),
      'RED': image.select('SR_B4'),
      'BLUE': image.select('SR_B2'),
      }).rename('EVI')
  return image.addBands(evi)

def get_landsat(lat, lon, start, end):
  point = ee.Geometry.Point(lon, lat)
  landsat8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterDate(start, end).filterBounds(point)
  landsat9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2").filterDate(start, end).filterBounds(point)
  landsat = landsat8.merge(landsat9).map(cloudMask).map(addEVI).median().select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'EVI', 'ST_B10'])

  median = landsat.reduceRegion(
        reducer=ee.Reducer.median(),
        geometry=point,
        scale=30
    )

  return median

In [ ]:
dataframe = pd.DataFrame(columns=['latitude', 'longitude', 'year', 'vegstructure', 'SOC', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'EVI', 'ST_B10'])

for i in range(len(df)):
  print(i)
  for j in range(8):
    elem = get_landsat(df.iloc[i, 0], df.iloc[i, 1], 
                       ee.Date(str(df.iloc[i, 2]) + '-01-01').advance(-1, 'year').advance(3*j, 'month'), 
                       ee.Date(str(df.iloc[i, 2]), '-03-31').advance(-1, 'year').advance(3*j, 'month'))

    newRow = pd.Series( df.iloc[i, :5].to_list() + [ elem.get('SR_B2'), elem.get('SR_B3'), elem.get('SR_B4'), elem.get('SR_B5'), elem.get('SR_B6'), elem.get('SR_B7'), elem.get('EVI'), elem.get('ST_B10')],
                       index=dataframe.columns )
    dataframe = dataframe.append(newRow, ignore_index=True)
dataframe

In [ ]:
dataframe[dataframe.isna().any(axis=1)]

In [ ]:
dataframe.to_csv('/content/drive/My Drive/SOClandsat.csv', index=False)